# Data Understanding

![Screenshot](Screenshot.png)


#  Import Necessary Liberaries

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns


In [ ]:
# To Show All Column
pd.set_option('display.max_columns', None)

In [ ]:
# I took a sample of Data 
# df = pd.read_csv('flights_sample_3m.csv').sample(1000000).sort_values(by= "FL_DATE", ignore_index= True)
# df.to_csv('flights_sample_1m.csv', index= False)

# Data Loading

In [ ]:
import pandas as pd
import streamlit as st

url = "https://www.dropbox.com/scl/fi/ms75gkptiv8unvoyfsvkd/flights_sample_1m.csv?rlkey=et96egb6z67wll1avmu9vhnx6&st=vhduo8a0&dl=1"

df = pd.read_csv(url)

df

# Data Exploration

In [ ]:
# Check Data Type
df.info()

In [ ]:
df.head()

In [ ]:
# Check Summary Statistics For Numerical Columns
df.describe(include= "number").round(2)

In [ ]:
# Check Summary Statistics For Categorical Columns
df.describe(include= "O").round(2)

In [ ]:
# Check Duplicates
df.duplicated().sum()

In [ ]:
# Check Missing Value
df.isna().mean() * 100

In [ ]:
round((1 - df.dropna().shape[0] / df.shape[0]) * 100, 2) # All Rows has Missing Value (NaN)

# Data Cleaning

In [ ]:
# set the columns small letter
df.columns = df.columns.str.lower()
df.columns

In [ ]:
# Drop unnecessary Column
df.drop(columns= ['airline_dot', 'dot_code', 'airline_code','fl_number'], axis=1, inplace= True)

In [ ]:
df.duplicated().sum()

In [ ]:
# Convert fl_date to DateTime
df['fl_date'] = pd.to_datetime(df['fl_date'])
df['fl_date']

# In Depth Check For Categorical Columns

In [ ]:
cat_col = df.select_dtypes(include= "O").columns
cat_col

In [ ]:
for col in cat_col:
    print(col)
    print(df[col].nunique())
    print(df[col].unique())
    print("-" * 100)

# In Depth Check For Numerical Columns

In [ ]:
df.head()

In [ ]:
timing_columns = ['crs_dep_time', 'dep_time', 'crs_arr_time', 'arr_time', 'wheels_off', 'wheels_on']
timing_columns

In [ ]:
# Quik Check inconsistent value in Timing columns Without using Distribution because the data is very large
for col in timing_columns:
    print(col)
    print(df[col].min())
    print(df[col].max())
    print("-" * 50)

In [ ]:
num_col = df.select_dtypes(include= "number").drop(columns= timing_columns, axis= 1).columns
num_col

In [ ]:
for col in num_col:
    px.histogram(df, x= col, title= col).show()

In [ ]:
df.isna().mean().round(4) * 100

In [ ]:
# Missing value with flight canccelled
df[df['cancelled'] == 0][df.columns].isna().sum()

In [ ]:
# Most of the Missing value is due to cancelled flights
df[df['cancelled'] == 1]

# Feature Engineering

In [ ]:
# Extract a new column More clear for CANCELLATION_CODE
def cancellation_reson(x):
    if pd.isna(x) :
        return 'Not Cancelled'
    elif x == 'D' :
        return 'Security'
    elif x == 'B':
        return 'Weather'
    elif x == 'C':
        return 'NAS'
    else:
        return 'Airline'
df['cancellation_reason'] = df['cancellation_code'].apply(cancellation_reson)
df['cancellation_reason']

In [ ]:
# Extract Year
df['year'] = df.fl_date.dt.year
df['year']

In [ ]:
# Extract Month
df['month'] = df.fl_date.dt.strftime('%b')
df['month']

In [ ]:
# Extract Day
df['day'] = df.fl_date.dt.strftime('%a')
df['day']

In [ ]:
# Extract Flight Status
df['flight_status'] = df['cancelled'].apply(lambda x: 'Cancelled' if x == 1 else 'Completed')
df['flight_status']

In [ ]:
# Extract the flight diverted or not
df['diverted_status'] = df['diverted'].apply(lambda x: 'Diverted' if x == 1 else 'Not Diverted')
df['diverted_status']

In [ ]:
# Extract the dep time period
def extract_dep_time_period(x):
    if pd.isna(x):
        return 'Unknown'
    hour = int(x) // 100
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

df['dep_time_Period'] = df['dep_time'].apply(extract_dep_time_period)
df['dep_time_Period']

In [ ]:
# Extract the arr time period
def extract_arr_time_period(x):
    if pd.isna(x):
        return 'Unknown'
    hour = int(x) // 100
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

df['arr_time_Period'] = df['arr_time'].apply(extract_arr_time_period)
df['arr_time_Period']

In [ ]:
df.head()

# Data Analysis

### Univariate Categorical

#### 1. What are the most frequent airlines ?

In [ ]:
airline_counts = df['airline'].value_counts(ascending= True).reset_index()
airline_counts.columns = ['AirLine', 'Number of Flights']
airline_counts

In [ ]:
px.bar(airline_counts, x='Number of Flights', y='AirLine', title='Most Frequent Airlines by Number of Flights', text_auto= True, height=600, width=1200)

#### 2. Which origin cities have the highest number of departing flights?

In [ ]:
origin_counts = df['origin_city'].value_counts().reset_index()
origin_counts.columns = ['ORIGIN CITY', 'Number of Flights']
origin_counts

In [ ]:
px.bar(origin_counts.head(10), x='ORIGIN CITY', y='Number of Flights', title='Top 10 Origin Cities by Number of Flights')

#### 3. Which destination cities have the highest number of arriving flights?



In [ ]:
dest_counts = df['dest_city'].value_counts().reset_index()
dest_counts.columns = ['DEST CITY', 'Number of Flights']
dest_counts

In [ ]:
px.bar(dest_counts.head(10), x='DEST CITY', y='Number of Flights', title='Top 10 Destination Cities by Number of Flights')

#### 4. What is the precentege flights were cancelled or not?

In [ ]:
px.pie(df, names='flight_status', title='Cancelled vs Not Cancelled Flights', hole= 0.5)

#### 5. What are the main reasons for flight cancellations?


In [ ]:
flight_cancelled = df[df['cancelled'] == 1]
flight_cancelled

In [ ]:
px.pie(flight_cancelled, names='cancellation_reason', title='Main Reasons for Flight Cancellations', hole= 0.5)


#### 6. What are the most common departure time periods (Morning, Afternoon, Evening, Night)?

In [ ]:
px.pie(df, names='dep_time_Period', title='Flight Distribution by Time Period', hole= 0.5)

### Univariate Numerical

#### 7. What is the distribution of scheduled flight duration (crs_elapsed_time)?

In [ ]:
px.histogram(df, x='crs_elapsed_time', nbins=100, title='Distribution of Scheduled Flight Duration', labels={'crs_elapsed_time': 'Scheduled Flight Duration (minutes)'})

#### 8. How far do most flights travel (DISTANCE)?

In [ ]:
px.histogram(df, x='distance',nbins=100, title='Distribution of Flight Distance', labels={'distance': 'Flight Distance (miles)'})

### Bivariate 
- Categorical vs Categorical

#### 9. Which airlines have the highest cancellation rates?

In [ ]:
airline_cancel_rate = df.groupby('airline')['cancelled'].mean().reset_index()
airline_cancel_rate = airline_cancel_rate.sort_values(by='cancelled')
airline_cancel_rate

In [ ]:
px.bar(airline_cancel_rate, x='cancelled', y='airline', title='Airline Cancellation Rates',
        labels={'cancelled': 'Cancellation Rate', 'airline': 'AirLine'}, text_auto= True, height=600, width=1200)

#### 10. Which month have the highest cancellation rates?

In [ ]:
month_cancel_rate = df.groupby('month')['cancelled'].mean().reset_index()
month_cancel_rate = month_cancel_rate.sort_values(by='cancelled')
month_cancel_rate

In [ ]:
px.bar(month_cancel_rate, x='cancelled', y='month', title='Month Cancellation Rates',
        labels={'cancelled': 'Cancellation Rate', 'month': 'Month'}, text_auto= True, height=600, width=1200)

- Categorical vs Numerical

#### 11. What is the average departure delay by airline?

In [ ]:
avg_dep_delay = df.groupby(['airline'])['dep_delay'].mean().reset_index().sort_values(by= 'dep_delay')
avg_dep_delay

In [ ]:
px.bar(avg_dep_delay, x= "dep_delay", y= 'airline', text_auto= True, height=600, width=1200, title='Average Departure Delay by Airline',
             labels={'dep_delay': 'Average Delay (minutes)', 'airline': 'Airline'},)

#### 12. How does flight distance vary by airline?

In [ ]:
avg_distance = df.groupby('airline')['distance'].mean().reset_index().sort_values(by= 'distance' , ascending= False)
avg_distance

In [ ]:
px.bar(avg_distance, x='airline', y='distance',
             title='Average Flight Distance per Airline',
             labels={'airline': 'Airline', 'distance': 'Avg Distance (miles)'})

#### 13. Which city pairs have the longest arr delays?

In [ ]:
top_delays = df[df['cancelled'] == 0].groupby(['origin_city', 'dest_city'])['arr_delay'].mean().reset_index().sort_values(by='arr_delay', ascending=False).head(10)
top_delays['city_pair'] = top_delays['origin_city'] + " ---> " + top_delays['dest_city']
top_delays

In [ ]:
px.bar(top_delays, x='city_pair', y='arr_delay', title='Top 10 City Pairs with Longest Arrival Delays', labels={'arr_delay': 'Average Arrival Delay (minutes)', 'city_pair': 'City Pair'})

### Multivariate

#### 14. what is the highest flight counts vary by airline, year, and destination city?

In [ ]:
flight_counts = df.groupby(['airline', 'year', 'dest_city']).size().reset_index(name='flight_count').sort_values(by= 'flight_count', ascending= False).head(10)
flight_counts


In [ ]:
px.bar(
    flight_counts,
    x='dest_city',
    y='flight_count',
    color='airline',
    facet_col='year',
    title='Top 10 Destinations by Airline and Year',
    labels={'dest_city': 'Destination City', 'flight_count': 'Flight Count'},
    height=600
)



#### 15. Which airlines operate longer routes on specific days of the week?

In [ ]:
avg_distance = df.groupby(['airline', 'day'])['distance'].mean().reset_index().sort_values(by= 'distance',ascending= False)
avg_distance

In [ ]:
fig = px.scatter(
    avg_distance,
    x='day',
    y='distance',
    color='airline',
    size='distance',
    title='Flight distance by airline and Day of the Week',
    labels={'day': 'Day of Week', 'distance': 'Average Distance (miles)'}
)
fig.show()


In [ ]:
px.bar(avg_distance, x='day', y='distance', color='airline', barmode='group', title='Average Flight distance by Airline and Day of the Week',
       labels={'day': 'Day of Week', 'distance': 'Average Distance (miles)'}, width= 1500)

In [ ]:
df.head()

# Deployment (streamlit)

In [ ]:
# Save Cleaned Dataframe
df.to_csv('Flight_Canselled_Delay_C.csv', index= False)

In [ ]:
import streamlit as st

In [ ]:
%%writefile Home.py

import streamlit as st

st.set_page_config(page_title="Flight Delay Dashboard", layout="wide", page_icon="✈️")

# Title
st.markdown("""<h1 style='text-align: center; color: #003049; font-size: 50px;'>
                    ✈️ Flight Delay and Cancellation Analysis
                    <h1 style='text-align: center; color: #003049; 'font-size: 50px; color: #003049;'>
                        (2019 - 2023)
                    </h1>
                </h1>""", unsafe_allow_html=True)

# Description
st.markdown("""
    <div style='background-color: #c7d9be; padding: 25px; border-radius: 12px;'>
        <p style='font-size: 18px; color: #264653;'>
            Welcome to the <strong>Flight Delay and Cancellation Analysis Dashboard</strong>.<br>
            This dashboard explores U.S. Flight data from 2019 to 2023 to cover insights related to:
        </p>
        <ul style='font-size: 17px; color: #2a9d8f;'>
            <li>🛫 Airport activity</li>
            <li>⏱️ Flight delays</li>
            <li>❌ Cancellations</li>
            <li>📊 Airline performance</li>
            <li>📌 And more...</li>
        </ul>
        <p style='font-size: 16px; color: #264653;'>
            Use the sidebar to explore interactive visualizations and uncover insights that help understand U.S. flight trends and challenges.
        </p>
    </div>
""", unsafe_allow_html=True)

# Navigation Links 
st.markdown("### 📂 Explore the Dashboard")
st.page_link("pages/1-Flight_Overview.py", label="📈 Flight Overview")
st.page_link("pages/2-Airline_Analysis.py", label="🛩️ Airline Analysis")
st.page_link("pages/3-Airport_Analysis.py", label="🛬 Airport Analysis")
st.page_link("pages/4-Project_Presentation.py", label="🗂️ Project Presentation")
st.page_link("pages/5-About.py", label="👤 About Me")

# About Me Section
st.markdown("---")
st.markdown("### 👨‍💻 About the Developer")
st.markdown("""
            - **Name:** Ahmed Shlaby  
            - **GitHub:** [github.com/ahmedshlaby](https://github.com/ahmedshlaby)  
            - **LinkedIn:** [linkedin.com/in/ahmedshlaby](https://linkedin.com/in/ahmedshlaby)  
            - **Email:** [shalabyahmed299@gmail.com](mailto:shalabyahmed299@gmail.com)
            """)


# Footer
st.markdown("""---""")
st.markdown("""
                <p style='text-align: center; font-size: 14px;'>
                    © 2025 | Developed by <strong>Ahmed Shlaby</strong> | 📧 <a href="mailto:shalabyahmed299@gmail.com">Contact</a>
                </p>
            """, unsafe_allow_html=True)


In [ ]:
%%writefile pages/1-Flight_Overview.py


import streamlit as st
import pandas as pd
import plotly.express as px
from PIL import Image

# Function to format large numbers
def format_number(n):
    if n >= 1_000_000:
        return f"{n/1_000_000:.1f}M"
    elif n >= 1_000:
        return f"{n/1_000:.1f}K"
    else:
        return str(n)


# Page configuration
st.set_page_config(page_title="Flight Analysis Dashboard", page_icon="✈️", layout="wide", initial_sidebar_state="expanded")


# CSS style f
st.markdown("""
    <style>
    .kpi-card {
        background-color: #c7d9be;
        border-radius: 16px;
        padding: 20px;
        text-align: center;
        box-shadow: 0 4px 10px rgba(0, 0, 0, 0.05);
        transition: transform 0.2s ease-in-out;
        margin: 10px;
    }
    .kpi-card:hover {
        transform: scale(1.02);
        background-color: #5e8d83;
        color: #fff;
    }
    .kpi-label {
        font-size: 16px;
        font-weight: 600;
        color: #2f3e46;
        margin-bottom: 8px;
    }
    .kpi-value {
        font-size: 34px;
        font-weight: 800;
        color: #1b4332;
        margin-bottom: 5px;
    }
    .kpi-delta {
        font-size: 14px;
        font-weight: 500;
        color: #6c757d;
    }
    .kpi-card:hover .kpi-delta {
        color: #e6f4ea;
    }
    </style>
""", unsafe_allow_html=True)


# Load dataset
@st.cache_data
def load_data():
    url = "https://www.dropbox.com/scl/fi/cdrfwk27h6sszbqg2k82b/Flight_Canselled_Delay_C.csv?rlkey=0nnticgct444wwqqjk50ctov4&st=aazxpeja&dl=1"
    df = pd.read_csv(url, parse_dates=["fl_date"])
    return df

df = load_data()

# Sidebar filters
st.sidebar.markdown("## 🧭 Filters")
st.sidebar.markdown("---")

# Date Range Selector
min_date = df['fl_date'].min().date()
max_date = df['fl_date'].max().date()

start_date, end_date = st.sidebar.date_input("📅 Select Date Range:", value=(min_date, max_date), min_value=min_date, max_value=max_date)

selected_airline = st.sidebar.multiselect("Select Airline(s):", options=sorted(df['airline'].unique()), default=[])
selected_origin = st.sidebar.multiselect("Select Origin City:", options=sorted(df['origin_city'].unique()), default=[])
selected_dest = st.sidebar.multiselect("Select Destination City:", options=sorted(df['dest_city'].unique()), default=[])
selected_status = st.sidebar.multiselect("Select Flight Status:", options=sorted(df['flight_status'].unique()), default=[])

# Apply filters
df_filtered = df[(df['fl_date'] >= pd.to_datetime(start_date)) & (df['fl_date'] <= pd.to_datetime(end_date))]

if selected_airline:
    df_filtered = df_filtered[df_filtered['airline'].isin(selected_airline)]
if selected_origin:
    df_filtered = df_filtered[df_filtered['origin_city'].isin(selected_origin)]
if selected_dest:
    df_filtered = df_filtered[df_filtered['dest_city'].isin(selected_dest)]
if selected_status:
    df_filtered = df_filtered[df_filtered['flight_status'].isin(selected_status)]

# Overview 
st.markdown("## 📊 Overview Dashboard")

# KPIs
col1, col2, col3, col4 = st.columns(4)

total_flights = df_filtered.shape[0]
delayed_flights = df_filtered[df_filtered['dep_delay'] > 15].shape[0]
cancelled_flights = df_filtered[df_filtered['flight_status'] == 'Cancelled'].shape[0]
on_time_flights = total_flights - delayed_flights - cancelled_flights

delay_percent = (delayed_flights / total_flights) * 100 if total_flights > 0 else 0
cancel_percent = (cancelled_flights / total_flights) * 100 if total_flights > 0 else 0


with col1:
    st.markdown(f"""
        <div class="kpi-card">
            <div class="kpi-label">✈️ Total Flights</div>
            <div class="kpi-value">{format_number(total_flights)}</div>
        </div>
    """, unsafe_allow_html=True)


with col2:
    st.markdown(f"""
        <div class="kpi-card">
            <div class="kpi-label">🕒 Delayed Flights</div>
            <div class="kpi-value">{format_number(delayed_flights)}</div>
            
        </div>
    """, unsafe_allow_html=True)

with col3:
    st.markdown(f"""
        <div class="kpi-card">
            <div class="kpi-label">❌ Cancelled Flights</div>
            <div class="kpi-value">{format_number(cancelled_flights)}</div>
            
        </div>
    """, unsafe_allow_html=True)


with col4:
    st.markdown(f"""
        <div class="kpi-card">
            <div class="kpi-label">✅ On-Time Flights</div>
            <div class="kpi-value">{format_number(on_time_flights)}</div>
        </div>
    """, unsafe_allow_html=True)


st.markdown("---")


# Color Palette 
custom_colors = ["#114538", "#5e8d83", "#d2e1cc", "#161d23"]

# Flights Over Time
st.markdown("## 📅 Flights Over Time")
chart_type = st.radio("Select Chart Type:", ["Line Chart", "Bar Chart"], horizontal=True)

df_filtered["year"] = df_filtered["fl_date"].dt.year
flights_over_time = df_filtered.groupby("year").size().reset_index(name="count")

if chart_type == "Line Chart":
    st.plotly_chart(px.line(flights_over_time, x="year", y="count", markers=True,
                  title="Flights Count Over Time",
                  labels={"year": "Year", "count": "Number of Flights"},
                  template="plotly_white", color_discrete_sequence=custom_colors), use_container_width=True)
else:
    st.plotly_chart(px.bar(flights_over_time, x="year", y="count",
                 title="Flights Count Over Time",
                 labels={"year": "Year", "count": "Number of Flights"},
                 template="plotly_white", color_discrete_sequence=custom_colors), use_container_width=True)

# Flight Status Distribution
st.markdown("## 📌 Flight Status Distribution")

status_counts = df_filtered['flight_status'].value_counts().reset_index()
status_counts.columns = ['Status', 'Count']

st.plotly_chart(px.pie(status_counts, names='Status', values='Count',
              title="Flight Status Breakdown", hole = 0.5,
              color_discrete_sequence=['#114538', '#5e8d83', '#d2e1cc']), use_container_width=True)



# Footer
st.markdown("""---""")
st.markdown("""
    <p style='text-align: center; font-size: 14px;'>
        © 2025 | Developed by <strong>Ahmed Shlaby</strong> | 📧 <a href="mailto:shalabyahmed299@gmail.com">Contact</a>
    </p>
""", unsafe_allow_html=True)


In [ ]:
%%writefile pages/2-Airline_Analysis.py

import streamlit as st
import pandas as pd
import plotly.express as px

# Page configuration
st.set_page_config(page_title="Airline Insights", page_icon="📈", layout="wide")

# Load dataset
@st.cache_data
def load_data():
    url = "https://www.dropbox.com/scl/fi/cdrfwk27h6sszbqg2k82b/Flight_Canselled_Delay_C.csv?rlkey=0nnticgct444wwqqjk50ctov4&st=aazxpeja&dl=1"
    df = pd.read_csv(url, parse_dates=["fl_date"])
    return df

df = load_data()

# Sidebar filters
st.sidebar.markdown("## ✈️ Airline Insights Filters")
st.sidebar.markdown("---")

min_date = df['fl_date'].min().date()
max_date = df['fl_date'].max().date()

start_date, end_date = st.sidebar.date_input("📅 Select Date Range:", value=(min_date, max_date), min_value=min_date, max_value=max_date)

selected_airline = st.sidebar.multiselect("Select Airline(s):", options=sorted(df['airline'].unique()), default=[])
selected_status = st.sidebar.multiselect("Select Flight Status:", options=sorted(df['flight_status'].unique()), default=[])

# Filter data
df_filtered = df[(df['fl_date'] >= pd.to_datetime(start_date)) & (df['fl_date'] <= pd.to_datetime(end_date))]

if selected_airline:
    df_filtered = df_filtered[df_filtered['airline'].isin(selected_airline)]
if selected_status:
    df_filtered = df_filtered[df_filtered['flight_status'].isin(selected_status)]


# tabs for choise
tab1, tab2, tab3, tab4 = st.tabs(["📊 Overview", "❌ Cancellation Analysis", "🕒 Delay Analysis", "🔁 Compare Airlines"])


# Top Airlines by Count (overview tab)
with tab1:
    st.markdown("## 🏆 Top 10 Airlines by Flight Count")
    top_airlines = df_filtered['airline'].value_counts().nlargest(10).reset_index()
    top_airlines.columns = ['Airline', 'Flights']

    st.plotly_chart(px.bar(top_airlines, x='Airline', y='Flights',
                title="Top 10 Airlines by Number of Flights",
                color='Flights', color_continuous_scale='Tealgrn',
                template='plotly_white'), use_container_width=True)

    # Flights Distribution by Time of Day
    st.markdown("## ☀️ Flights Distribution by Time of Day")

    dep_period_counts = df_filtered.groupby(['airline', 'dep_time_Period']).size().reset_index(name='flight_count').sort_values(by= 'flight_count', ascending= False)

    st.plotly_chart(px.bar(dep_period_counts,
                x='airline',
                y='flight_count',
                color='dep_time_Period',
                title='Flights by Time of Day and Airline',
                barmode='group',
                labels={'flight_count': 'Number of Flights', 'dep_time_Period': 'Time Period'},
                template='plotly_white',
                color_discrete_sequence=px.colors.sequential.Tealgrn), use_container_width=True)

    

# Cancellation Rate by Airline
with tab2:
    st.markdown("## ❌ Cancellation Rate by Airline")
    tab1, tab2 = st.tabs(["🥧 Pie Chart", "📊 Bar Chart"])

    cancel_data = df_filtered.groupby("airline").agg(
        total_flights=('flight_status', 'count'),
        cancelled_flights=('flight_status', lambda x: (x == 'Cancelled').sum())).reset_index()

    cancel_data['cancellation_rate'] = (cancel_data['cancelled_flights'] / cancel_data['total_flights']) * 100

    # Monthly Cancellation Rate per Airline
    st.markdown("## 📉 Monthly Cancellation Rate per Airline")

    df_filtered['month'] = df_filtered['fl_date'].dt.to_period('M').astype(str)
    monthly_cancel = df_filtered.groupby(['month', 'airline'])['cancelled'].mean().reset_index()

    st.plotly_chart(px.line(monthly_cancel, x='month', y='cancelled', color='airline',
                title='Monthly Cancellation Rate per Airline',
                labels={'cancelled': 'Cancellation Rate'},
                template='plotly_white', color_discrete_sequence=px.colors.sequential.Tealgrn), use_container_width=True)


    with tab1:  # Pie Chart
        st.plotly_chart(
            px.pie(cancel_data, names="airline", values="cancellation_rate",
                title="Pie Chart: Cancellation Rate Distribution",
                color_discrete_sequence=px.colors.sequential.Tealgrn), use_container_width=True)


    with tab2:  # Bar Chart
        st.plotly_chart(
            px.bar(cancel_data.sort_values(by="cancellation_rate", ascending=False),
                x="airline", y="cancellation_rate",
                title="Bar Chart: Cancellation Rate by Airline (%)",
                labels={"airline": "Airline", "cancellation_rate": "Cancellation Rate (%)"},
                color="cancellation_rate", color_continuous_scale='Tealgrn',
                template="plotly_white"), use_container_width=True)




# Average Delay by Airline
with tab3:
    st.markdown("## 🕒 Average Departure Delay by Airline")

    avg_delay = df_filtered.groupby('airline')['dep_delay'].mean().reset_index().sort_values(by='dep_delay', ascending=False)

    st.plotly_chart(px.bar(avg_delay, x='airline', y='dep_delay',
                title='Average Departure Delay (in minutes)',
                labels={'airline': 'Airline', 'dep_delay': 'Avg Departure Delay'},
                color='dep_delay', color_continuous_scale='Tealgrn',
                template='plotly_white'), use_container_width=True)
    
    # for arr delay
    st.markdown("## 🕒 Average Arrival Delay by Airline")

    avg_delay = df_filtered.groupby('airline')['arr_delay'].mean().reset_index().sort_values(by='arr_delay', ascending=False)

    st.plotly_chart(px.bar(avg_delay, x='airline', y='arr_delay',
                title='Average Departure Delay (in minutes)',
                labels={'airline': 'Airline', 'arr_delay': 'Avg Departure Delay'},
                color='arr_delay', color_continuous_scale='Tealgrn',
                template='plotly_white'), use_container_width=True)


# compare between 2 Airline
with tab4:
    st.markdown("## ✈️ Compare Airlines")

    airlines_to_compare = st.multiselect(
        "Select up to 2 Airlines to Compare:",
        options=df_filtered['airline'].unique(),
        default=df_filtered['airline'].unique()[:2],
        max_selections=2
    )

    if len(airlines_to_compare) == 2:
        compare_df = df_filtered[df_filtered['airline'].isin(airlines_to_compare)]

        # Total per airlinee
        compare_grouped = compare_df.groupby('airline').agg(
            total_flights=('flight_status', 'count'),
            cancelled_flights=('flight_status', lambda x: (x == 'Cancelled').sum()),
            avg_delay=('dep_delay', 'mean')
        ).reset_index()

        compare_grouped['cancellation_rate'] = (compare_grouped['cancelled_flights'] / compare_grouped['total_flights']) * 100

        # Kpis
        st.markdown("### 📌 Key Metrics")
        col1, col2 = st.columns(2)

        with col1:
            st.metric(label=f"{airlines_to_compare[0]} - Total Flights", value=compare_grouped.loc[0, 'total_flights'])
            st.metric(label=f"{airlines_to_compare[0]} - Cancellation Rate", value=f"{compare_grouped.loc[0, 'cancellation_rate']:.2f}%")
            st.metric(label=f"{airlines_to_compare[0]} - Avg Delay", value=f"{compare_grouped.loc[0, 'avg_delay']:.2f} min")

        with col2:
            st.metric(label=f"{airlines_to_compare[1]} - Total Flights", value=compare_grouped.loc[1, 'total_flights'])
            st.metric(label=f"{airlines_to_compare[1]} - Cancellation Rate", value=f"{compare_grouped.loc[1, 'cancellation_rate']:.2f}%")
            st.metric(label=f"{airlines_to_compare[1]} - Avg Delay", value=f"{compare_grouped.loc[1, 'avg_delay']:.2f} min")

        # Comparison Table
        st.markdown("### 📊 Comparison Table")
        st.dataframe(compare_grouped)

        # charts
        st.markdown("### 📈 Comparison Charts")
        st.plotly_chart(
            px.bar(compare_grouped, x='airline', y='total_flights',
                title="Total Flights per Airline", color='total_flights',
                color_continuous_scale='Tealgrn', template='plotly_white',
                labels = {'airline' : 'AirLine', 'total_flights': 'Number Of Flights'},
                text_auto=True),
            use_container_width=True)

        st.plotly_chart(
            px.bar(compare_grouped, x='airline', y='cancellation_rate',
                title="Cancellation Rate (%) per Airline", color='cancellation_rate',
                color_continuous_scale='Tealgrn', template='plotly_white',
                labels = {'airline' : 'AirLine', 'cancellation_rate': 'Cancellation Rate'},
                text_auto='.2f'),
            use_container_width=True)

        st.plotly_chart(
            px.bar(compare_grouped, x='airline', y='avg_delay',
                title="Average Delay (minutes) per Airline", color='avg_delay',
                color_continuous_scale='Tealgrn', template='plotly_white',
                labels = {'airline' : 'AirLine', 'avg_delay': 'Avg Delay (min)'},
                text_auto='.2f'),
            use_container_width=True)

        # Monthly Comparison
        st.markdown("### 🗓️ Monthly Comparison")

        monthly_compare = compare_df.copy()
        monthly_compare['month'] = monthly_compare['fl_date'].dt.to_period('M').astype(str)

        monthly_summary = monthly_compare.groupby(['month', 'airline']).agg(
            total_flights=('flight_status', 'count'),
            cancelled_flights=('flight_status', lambda x: (x == 'Cancelled').sum()),
            avg_delay=('dep_delay', 'mean')
        ).reset_index()

        monthly_summary['cancellation_rate'] = (monthly_summary['cancelled_flights'] / monthly_summary['total_flights']) * 100

        st.plotly_chart(px.line(monthly_summary, x='month', y='total_flights', color='airline',
                      title='Monthly Flight Count per Airline',
                      labels = {'month' : 'Month (year)', 'total_flights': 'Total Flights'},
                      template='plotly_white'), use_container_width=True)
        
        
        # Analysis insight 
        st.markdown("### 🧠 Insight")

        if compare_grouped.loc[0, 'avg_delay'] > compare_grouped.loc[1, 'avg_delay']:
            worst_airline = compare_grouped.loc[0, 'airline']
        else:
            worst_airline = compare_grouped.loc[1, 'airline']

        st.info(f"📌 Based on the current data, **{worst_airline}** has a higher average delay.")




# Footer
st.markdown("""---""")
st.markdown("""
    <p style='text-align: center; font-size: 14px;'>
        © 2025 | Developed by <strong>Ahmed Shlaby</strong> | 📧 <a href="mailto:shalabyahmed299@gmail.com">Contact</a>
    </p>
""", unsafe_allow_html=True)



In [ ]:
%%writefile pages/3-Airport_Analysis.py

import streamlit as st
import pandas as pd
import plotly.express as px

# Page configuration
st.set_page_config(page_title="Airport Analysis", page_icon="🛫", layout="wide")
st.title("🛫 Airport Analysis")

# Load dataset
@st.cache_data
def load_data():
    url = "https://www.dropbox.com/scl/fi/cdrfwk27h6sszbqg2k82b/Flight_Canselled_Delay_C.csv?rlkey=0nnticgct444wwqqjk50ctov4&st=aazxpeja&dl=1"
    df = pd.read_csv(url, parse_dates=["fl_date"])
    return df

df = load_data()

# Sidebar - Airport selection
st.sidebar.header("✈️ Filter Airports")
airport_options = df['origin'].value_counts().head(20).index.tolist()
selected_airports = st.sidebar.multiselect("Select Origin Airports", airport_options, default=[])




if selected_airports:
    df = df[df['origin'].isin(selected_airports)]


# Tabs for different analyses
tab1, tab2, tab3, tab4 = st.tabs(["📊 Overview", "❌ Cancellation Analysis", "⏱️ Delay Analysis", "🔍 Compare Airports"])

with tab1:
    st.subheader("Top 10 Crowded Airports")
    origin_counts = df['origin'].value_counts().head(10).reset_index()
    origin_counts.columns = ['Origin Airport', 'Flight Count']
    st.plotly_chart(px.bar(origin_counts, x='Origin Airport', y='Flight Count', 
                    color_continuous_scale='Tealgrn', color= 'Flight Count'), use_container_width=True)


with tab2:
    st.subheader("Cancellation Rate by Airport")
    cancel_rate = df.groupby('origin')['cancelled'].mean().sort_values(ascending=False).head(10).reset_index()
    cancel_rate.columns = ['Origin Airport', 'Cancellation Rate']
    st.plotly_chart(px.bar(cancel_rate, x='Origin Airport', y='Cancellation Rate', title='Top 10 Airports by Cancellation Rate',
                     color='Cancellation Rate', color_continuous_scale='Tealgrn'), use_container_width=True)


with tab3:
    st.subheader("Top 10 Airports by Average Departure Delay")
    dep_delay = df.groupby('origin')['dep_delay'].mean().sort_values(ascending=False).head(10).reset_index()
    dep_delay.columns = ['Origin Airport', 'Average Departure Delay']
    fig_dep = px.bar(dep_delay, x='Origin Airport', y='Average Departure Delay', 
                     color='Average Departure Delay', color_continuous_scale='Tealgrn',
                     title='Top 10 Airports by Average Departure Delay')
    st.plotly_chart(fig_dep, use_container_width=True)

    st.subheader("Top 10 Airports by Average Arrival Delay")
    arr_delay = df.groupby('dest')['arr_delay'].mean().sort_values(ascending=False).head(10).reset_index()
    arr_delay.columns = ['Destination Airport', 'Average Arrival Delay']
    fig_arr = px.bar(arr_delay, x='Destination Airport', y='Average Arrival Delay',
                     color='Average Arrival Delay', color_continuous_scale='Tealgrn',
                     title='Top 10 Airports by Average Arrival Delay')
    st.plotly_chart(fig_arr, use_container_width=True)


with tab4:
    st.subheader("Compare Two Airports")
    airport_compare = st.multiselect("Select Two Airports to Compare", df['origin'].unique(), default=df['origin'].unique()[:2])

    if len(airport_compare) == 2:
        airport_df = df[df['origin'].isin(airport_compare)]
        comp_stats = airport_df.groupby('origin').agg({
            'dep_delay': 'mean',
            'arr_delay': 'mean',
            'cancelled': 'mean'}).reset_index()

        st.plotly_chart(px.bar(
            comp_stats.melt(id_vars='origin', var_name='Metric', value_name='Value'),
            x='origin', y='Value', color='Metric', barmode='group',
            title='Airport Comparison: Cancellation Rate & Delay',
            color_discrete_sequence=['#005f73', '#0a9396', '#ee9b00']), use_container_width=True)


# Footer
st.markdown("""---""")
st.markdown("""
    <p style='text-align: center; font-size: 14px;'>
        © 2025 | Developed by <strong>Ahmed Shlaby</strong> | 📧 <a href="mailto:shalabyahmed299@gmail.com">Contact</a>
    </p>
""", unsafe_allow_html=True)





In [ ]:
%%writefile pages/4-Project_Presentation.py

import streamlit as st

st.set_page_config(page_title="Project Presentation", layout="wide")

# Title
st.markdown("""
    <h1 style='text-align: center; color: #003049; font-size: 42px; margin-bottom: 20px;'>
        📊 Flight Delay & Cancellation Project Presentation
    </h1>
""", unsafe_allow_html=True)

# Section 1: Data Understanding
st.header("1. Data Understanding")
st.markdown("""
- Loaded flight data from CSV containing records from 2019 to 2023.
- Data includes features such as flight date, origin, destination, delays, cancellation status, and airline info.
- Main goal: Analyze flight delay and cancellation patterns across US airports and airlines.
""")

# Section 2: Data Cleaning
st.header("2. Data Cleaning & Preparation")
st.markdown("""
- Converted `fl_date` column to datetime format.
- Extracted new features: `month` and `day_of_week` from the flight date.
- Removed irrelevant columns that do not contribute to prediction or cause data leakage, e.g., actual delay times, flight status, and city names.
- Removed duplicate rows.
- Dropped rows with missing values in important columns like `crs_elapsed_time`.
""")

# Section 3: Exploratory Data Analysis (EDA)
st.header("3. Exploratory Data Analysis (EDA)")
st.markdown("""
- Visualized key trends using interactive charts:
  - Top 10 busiest airports by number of flights.
  - Airports with highest cancellation rates.
  - Average departure and arrival delays by airports.
- Used bar charts, pie charts, and line charts for clear insights.
- This analysis helped to understand the data distribution and focus areas.
""")

# Section 4: Streamlit Dashboard
st.header("4. Interactive Dashboard with Streamlit")
st.markdown("""
- Built a multi-page Streamlit app to display analysis results.
- Pages include:
  - **Home:** Project overview and contact info.
  - **Flight Overview:** KPIs and overall flight trends.
  - **Airline Analysis:** Performance metrics by airline.
  - **Airport Analysis:** Deep dive into airport delays and cancellations.
- Used Plotly for dynamic and responsive visualizations.
- Applied consistent color schemes and user-friendly layout.
""")

# Section 5: Data Preprocessing for Modeling
st.header("5. Data Preprocessing for Machine Learning")
st.markdown("""
- Split data into features (`X`) and target (`y`) where target is `cancelled` (flight cancellation status).
- Performed train-test split (80% train, 20% test) with stratification to maintain class distribution.
- For numerical features:
  - Applied `RobustScaler` to normalize data and reduce the impact of outliers.
- For categorical features:
  - Used `BinaryEncoder` to convert categorical variables into numeric format.
""")

# Section 6: Handling Imbalanced Data
st.header("6. Handling Class Imbalance")
st.markdown("""
- The dataset is imbalanced: fewer cancelled flights compared to non-cancelled.
- Used **SMOTE (Synthetic Minority Oversampling Technique)** to generate synthetic samples for the minority class.
- This helps improve model training by balancing the classes and preventing bias toward the majority class.
""")

# foter
st.markdown("---")
st.markdown("""
<p style='text-align:center; font-size: 14px; color: #555;'>
    © 2025 | Developed by Ahmed Shlaby | 📧 <a href="mailto:shalabyahmed299@gmail.com">Contact Me</a>
</p>
""", unsafe_allow_html=True)




In [ ]:
%%writefile pages/5-About.py

import streamlit as st

# Page configuration
st.set_page_config(page_title="About Me", page_icon="👤", layout="centered")

# Page title
st.title("👤 About Me")

# Display profile picture 
st.image("MYIMAGE_Copy.jpg", width=200)

# introduction
st.markdown("""
Hi! I'm **Ahmed Shlaby**, a passionate Data Scientist and Python developer with a strong interest in flight data analysis and visualization.

---

### 🧑‍🎓 Background
- Bachelor's degree in Information Technology and Computer Science  
- Certified Data Scientist Professional (CDSP) from Epsilon AI  
- Certified Data Analysis Professional (CDAP) from Epsilon AI  
- Experienced in data analysis, visualization, and deploying interactive dashboards using Streamlit

---

### 🛠 Skills & Tools
- Python (Pandas, NumPy, Sklearn)  
- Data Visualization (Plotly, Seaborn, Matplotlib)  
- Dashboarding with Streamlit  
- Machine Learning basics & Data Preprocessing  
- GitHub for version control

---

### 📫 Contact Me
- **Email:** [shalabyahmed299@gmail.com](mailto:shalabyahmed299@gmail.com)  
- **LinkedIn:** [linkedin.com/in/ahmedshlaby](https://linkedin.com/in/ahmedshlaby)  
- **GitHub:** [github.com/ahmedshlaby](https://github.com/ahmedshlaby)  

---

Thank you for visiting my project! Feel free to reach out for collaboration or questions.
""")

# foter
st.markdown("---")
st.markdown("""
<p style='text-align:center; font-size: 14px; color: #555;'>
    © 2025 | Developed by Ahmed Shlaby | 📧 <a href="mailto:shalabyahmed299@gmail.com">Contact Me</a>
</p>
""", unsafe_allow_html=True)


In [ ]:
! streamlit run Home.py

In [ ]:
! pipreqs ./

# Preprocessing Phase

In [ ]:
df = pd.read_csv('Flight_Canselled_Delay_C.csv')

In [ ]:
# update columns month , day to numbers
df.fl_date = pd.to_datetime(df.fl_date)
df.fl_date

df['month'] = df.fl_date.dt.month
df['day_of_week'] = df.fl_date.dt.day_of_week


In [ ]:
# drop unnecessary columns
df.drop(columns= ['fl_date', 'origin_city', 'dest_city', 'wheels_off', 'wheels_on','cancellation_code', 'elapsed_time', 'flight_status',
                    'diverted_status', 'dep_time_Period', 'arr_time_Period', 'dep_time', 'arr_time', 'dep_delay', 'arr_delay',
                    'air_time', 'taxi_out', 'taxi_in', 'diverted', 
                    'delay_due_carrier', 'delay_due_weather',
                    'delay_due_nas', 'delay_due_security', 'delay_due_late_aircraft', 'cancellation_reason', 'day'], axis= 1, inplace= True)

In [ ]:
df.duplicated().sum()

In [ ]:
35701 / 1000000

In [ ]:
df.drop_duplicates(inplace= True, ignore_index= True)

In [ ]:
df.duplicated().sum()

In [ ]:
df.isna().sum()

In [ ]:
# drop nan value 
df.dropna(subset= 'crs_elapsed_time', ignore_index= True, inplace= True)

In [ ]:
df.duplicated().sum()

In [ ]:
df.head()

## Split Data To Input Features and Target 

In [ ]:
x = df.drop('cancelled', axis= 1)
y = df['cancelled']

## Split Data into Train and Test

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state= 1, stratify= y)

## Numerical

### impute missing value

In [ ]:
num_col = x_train.select_dtypes(include= 'number').columns
num_col

In [ ]:
x_test[num_col].isna().sum()

In [ ]:
x_train[num_col].isna().sum()

## Feature Scaling

In [ ]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

x_train[num_col] = scaler.fit_transform(x_train[num_col])
x_test[num_col] = scaler.transform(x_test[num_col])

In [ ]:
x_train.head()

In [ ]:
x_test.head()

## Categorical

In [ ]:
x_train.select_dtypes(include= "O").isna().sum()

### Nominal

In [ ]:
cat_col = x_train.select_dtypes(include= "O").columns
cat_col

In [ ]:
# check number of unique to explain which Encoder we use
for col in cat_col:
    print(col)
    print(df[col].nunique())
    print("-" * 50)


In [ ]:
# using Binary Encoder 
from category_encoders import BinaryEncoder

be = BinaryEncoder()

be_df_train = be.fit_transform(x_train[cat_col])

be_df_test = be.transform(x_test[cat_col])

In [ ]:
x_train = pd.concat([x_train, be_df_train], axis= 1).drop(x_train[cat_col], axis= 1)
x_test = pd.concat([x_test, be_df_test], axis= 1).drop(x_test[cat_col], axis= 1)


In [ ]:
x_train

In [ ]:
len(x_train)

In [ ]:
x_test

In [ ]:
len(x_test)

## Handle Imbalance

In [ ]:
y_train.value_counts(normalize=True) *100


In [ ]:
# using smote 
from imblearn.over_sampling import SMOTE

smote = SMOTE()

x_train_smote, y_train_smote = smote.fit_resample(x_train, y_train)


In [ ]:
y_train_smote.value_counts(normalize= True)